Various imports

In [ ]:
import scipy.io
import os, random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import requests
import tarfile

#np.set_printoptions(precision=2, linewidth=250)

Download and extract oxford validation dataset.
Training dataset could also be used but validation is lighter.

In [ ]:
# check if dataset has been downloaded: download it otherwise
validation_ds_tar_path = "validation_dataset.tar.gz"
if not(os.path.exists(validation_ds_tar_path)):
    response = requests.get("https://www.robots.ox.ac.uk/~vgg/data/hands/downloads/validation_dataset.tar.gz")
    open("validation_dataset.tar.gz", "wb").write(response.content)

# below there is copy paste of a code of the internet that checks whether dataset is fully extracted or not
directory = "."
def extract_nonexisting(archive):
    for name in archive.getnames():
        if not os.path.exists(os.path.join(directory, name)):
            archive.extract(name, path=directory)

archives = [name for name in os.listdir(directory) if name.endswith("tar.gz")]
for archive_name in archives:
    with tarfile.open(archive_name) as archive:
        extract_nonexisting(archive)

Now to list all images and randomly select one of them and it's relative annotation file

In [ ]:
# List files inside directory
img_list = os.listdir(path="validation_dataset/validation_data/images/")

# Randomly choose one
rnd = random.randint(0, len(img_list))
img_path = img_list[rnd]

# Now to select correct annotation
t = img_path.split('.')
annot_path = t[0] + '.mat'

# Delete useless values
del(img_list)
del(rnd)

# Attach full path
img_path = "validation_dataset/validation_data/images/" + img_path
annot_path = "validation_dataset/validation_data/annotations/" + annot_path

#print(img_path)
#print(annot_path)

In [ ]:
# loadmat() returns dict
mat = scipy.io.loadmat(annot_path)

# print contents of dict
#print("-------------------------------------------- FULL MAT CONTENT --------------------------------------------------------------")
#for key in mat:
#    print(key + " :  " + str(mat.get(key)))
#print("----------------------------------------------------------------------------------------------------------------------------")


Convert data to visualize it better in python

In [ ]:
# save points better for python
boxes = mat.get('boxes')

hand_boxes_accurate : list[np.ndarray] = []
for i in range(boxes.shape[1]):
    pts = np.zeros(shape=(4,2), dtype=int)
    for j in range(4):
        pts[j,0] = round(boxes[0,i][0,0][j][0,1])
        pts[j,1] = round(boxes[0,i][0,0][j][0,0])
    hand_boxes_accurate.append(pts)
    del(pts)


#print("------------------------------------------- ADAPTED MAT CONTENT ------------------------------------------------------------")
#for elem in hand_boxes_accurate:
#    print(elem)
#print("----------------------------------------------------------------------------------------------------------------------------")

Show image using matplotlib and draw rectangles on it

In [ ]:
# load image into ram
img = Image.open(img_path)

# create figure and axes
fig, ax = plt.subplots(dpi=220)

# display the image
ax.imshow(img)

# add accurate bounding boxes
for pts in hand_boxes_accurate:
    polygon = patches.Polygon(pts, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(polygon)

# compute and add relaxed bounding boxes
for pts in hand_boxes_accurate:
    origin_pt = np.amin(pts, axis=0)
    end_pt = np.amax(pts, axis=0)
    size = np.subtract(end_pt, origin_pt)
    
    rect = patches.Rectangle(tuple(origin_pt), size[0], size[1], linewidth=1, edgecolor='g', facecolor='none')
    ax.add_patch(rect)

plt.show()